In [1]:
#QFT ALGORITHM
import numpy as np
from qat.lang.AQASM import Program, H, CNOT, RZ
from qat.qpus import get_default_qpu

def qft(n):
    prog = Program()
    qbits = prog.qalloc(n)
    
    for i in range(n):
        prog.apply(H, qbits[i])
        for j in range(i+1, n):
            angle = np.pi / (2**(j-i))
            prog.apply(RZ(angle), qbits[j])
    
    return prog.to_circ()

# Set number of qubits for QFT
n = 3
circuit = qft(n)
qpu = get_default_qpu()
job = circuit.to_job(nbshots=1024)
result = qpu.submit(job)

# Display results
for sample in result:
    print(f"State: {sample.state}, Probability: {sample.probability}")


State: |000>, Probability: 0.12109375
State: |001>, Probability: 0.1142578125
State: |010>, Probability: 0.138671875
State: |011>, Probability: 0.11328125
State: |100>, Probability: 0.1318359375
State: |101>, Probability: 0.1318359375
State: |110>, Probability: 0.11328125
State: |111>, Probability: 0.1357421875


In [9]:
#Quantum Approximate Optimization Algorithm (QAOA)

import numpy as np
from qat.lang.AQASM import Program, H, CNOT, RX, RZ
from qat.qpus import get_default_qpu

# Define problem parameters
num_qubits = 3
gamma = np.pi / 4  # Angle for problem Hamiltonian
beta = np.pi / 8   # Angle for mixing Hamiltonian

# Create quantum program
prog = Program()
qbits = prog.qalloc(num_qubits)

# Apply Hadamard to all qubits
for q in qbits:
    prog.apply(H, q)

# Apply problem unitary (example for simple cost function)
prog.apply(CNOT, qbits[0], qbits[1])  # Entangling CNOT
prog.apply(RZ(gamma), qbits[1])       # Apply RZ rotation on qubit 1
prog.apply(CNOT, qbits[0], qbits[1])  # Entangling CNOT again

# Apply mixing unitary (RX rotations)
for q in qbits:
    prog.apply(RX(beta), q)

# Measure all qubits
for q in qbits:
    prog.measure(q)

# Compile and execute
circuit = prog.to_circ()
qpu = get_default_qpu()
job = circuit.to_job(nbshots=1024)  # Number of samples
result = qpu.submit(job)

# Display results
for sample in result:
    print(f"State: {sample.state}, Probability: {sample.probability}")


State: |000>, Probability: 0.185546875
State: |111>, Probability: 0.1748046875
State: |110>, Probability: 0.1953125
State: |001>, Probability: 0.1787109375
State: |101>, Probability: 0.0546875
State: |011>, Probability: 0.06640625
State: |100>, Probability: 0.0732421875
State: |010>, Probability: 0.0712890625


In [19]:
#shor algorithm
import numpy as np
from qat.lang.AQASM import Program, H, CNOT, RZ, RX
from qat.qpus import get_default_qpu
from math import gcd
import random

# Function to implement Quantum Fourier Transform (QFT)
def qft(prog, qbits):
    n = len(qbits)
    for i in range(n):
        prog.apply(H, qbits[i])
        for j in range(i+1, n):
            angle = np.pi / (2 ** (j - i + 1))
            prog.apply(RZ(angle), qbits[j])

# Function to implement inverse Quantum Fourier Transform (QFT†)
def qft_dagger(prog, qbits):
    n = len(qbits)
    for i in range(n-1, -1, -1):
        prog.apply(H, qbits[i])
        for j in range(i-1, -1, -1):
            angle = -np.pi / (2 ** (i - j + 1))
            prog.apply(RZ(angle), qbits[j])

# Function to simulate Shor's Algorithm
def shors_algorithm(N):
    # Step 1: Choose a random integer a such that 1 < a < N
    a = np.random.randint(2, N-1)

    # Step 2: Calculate gcd(a, N)
    if gcd(a, N) != 1:
        return gcd(a, N)

    # Step 3: Define number of qubits and set parameters for QFT
    num_qubits = 4  # Can be adjusted for more precision
    prog = Program()

    # Step 4: Initialize qubits and apply Hadamard gates
    qbits = prog.qalloc(num_qubits)
    for q in qbits:
        prog.apply(H, q)

    # Step 5: Apply modular exponentiation (function f(x) = a^x mod N)
    # Note: This part is done classically as Shor's algorithm requires a quantum circuit
    # for the period finding, not the full modular exponentiation
    # For simplicity, we won't do full modular exponentiation in this quantum simulation
    # However, this would typically be implemented using controlled-U gates

    # Step 6: Apply QFT (Quantum Fourier Transform) - Using the custom QFT implementation
    qft(prog, qbits)  # Apply QFT

    # Step 7: Measure qubits
    for q in qbits:
        prog.measure(q)

    # Step 8: Run the quantum circuit on the QPU
    circuit = prog.to_circ()
    qpu = get_default_qpu()
    job = circuit.to_job(nbshots=1024)
    result = qpu.submit(job)

    # Step 9: Analyze the results
    measured_states = []
    for sample in result:
        state = sample.state
        prob = sample.probability
        measured_states.append((state, prob))
        print(f"State: {state}, Probability: {prob}")

    # Step 10: Classical post-processing: Using continued fractions to find the period
    # We would extract the period (r) from the measurement results.
    # In the real-world scenario, we'd use the measured quantum output to find the period
    # using continued fractions, but for simplicity, we assume that this can be handled classically.

    # For now, let's assume we have found the period from the output.
    # In practice, the quantum output would lead to finding the period.
    # For example, let's assume the period is 4 (just as a demonstration).
    period = 4  # Normally, this is derived from the QFT result

    # Use the period (r) to find factors
    factor1 = gcd(a**(period // 2) - 1, N)
    factor2 = gcd(a**(period // 2) + 1, N)
    
    return f"Factors of {N} are {factor1} and {factor2}"

# Example usage: Try factoring 21
N = 21
print(shors_algorithm(N))


State: |0001>, Probability: 0.2490234375
State: |0100>, Probability: 0.068359375
State: |0010>, Probability: 0.1611328125
State: |0110>, Probability: 0.0205078125
State: |0000>, Probability: 0.34765625
State: |0011>, Probability: 0.0986328125
State: |0101>, Probability: 0.044921875
State: |0111>, Probability: 0.009765625
Factors of 21 are 3 and 1


In [21]:
#GROVER ALGORITHM
import numpy as np
from qat.lang.AQASM import Program, H, X, CNOT
from qat.qpus import get_default_qpu

# Function to implement Grover's Oracle
def grovers_oracle(prog, qbits, marked_state):
    """
    Grover's oracle applies a phase flip to the marked state.
    For simplicity, we assume the marked state is |11>.
    """
    if marked_state == '11':
        prog.apply(X, qbits[0])
        prog.apply(X, qbits[1])
    # Apply controlled-Z gate to the marked state (|11>)
    prog.apply(CNOT, qbits[0], qbits[1])
    prog.apply(CNOT, qbits[1], qbits[0])
    prog.apply(CNOT, qbits[0], qbits[1])

# Function to apply Grover's Diffusion Operator (Amplitude Amplification)
def grovers_diffusion_operator(prog, qbits):
    """
    The diffusion operator is used to amplify the probability of the marked state.
    """
    # Apply Hadamard to all qubits
    for q in qbits:
        prog.apply(H, q)
    
    # Apply X to all qubits (not to be confused with the Hadamard)
    for q in qbits:
        prog.apply(X, q)
    
    # Apply the controlled-Z gate
    prog.apply(CNOT, qbits[0], qbits[1])
    
    # Apply X to all qubits again
    for q in qbits:
        prog.apply(X, q)
    
    # Apply Hadamard to all qubits
    for q in qbits:
        prog.apply(H, q)

# Grover's Algorithm Implementation
def grovers_algorithm(N, marked_state):
    """
    Grover's algorithm for finding the marked state in a database of size N.
    The algorithm will apply the oracle and the diffusion operator iteratively.
    """
    # Step 1: Define number of qubits
    num_qubits = int(np.ceil(np.log2(N)))  # Calculate the number of qubits needed
    prog = Program()
    
    # Step 2: Initialize qubits (set to |0> state initially)
    qbits = prog.qalloc(num_qubits)
    
    # Step 3: Apply Hadamard gates to all qubits to create a superposition of all states
    for q in qbits:
        prog.apply(H, q)
    
    # Step 4: Apply Grover's Oracle and Diffusion Operator iteratively
    num_iterations = int(np.pi / 4 * np.sqrt(N))  # Optimal number of iterations for Grover's algorithm
    for _ in range(num_iterations):
        # Apply Oracle (phase flip on marked state)
        grovers_oracle(prog, qbits, marked_state)
        
        # Apply the Diffusion Operator (Amplitude Amplification)
        grovers_diffusion_operator(prog, qbits)
    
    # Step 5: Measure all qubits
    for q in qbits:
        prog.measure(q)
    
    # Step 6: Compile and execute the quantum circuit
    circuit = prog.to_circ()
    qpu = get_default_qpu()
    job = circuit.to_job(nbshots=1024)
    result = qpu.submit(job)
    
    # Step 7: Display the results (measured states and their probabilities)
    print("Grover's Algorithm Results:")
    measured_states = []
    for sample in result:
        state = sample.state
        prob = sample.probability
        measured_states.append((state, prob))
        print(f"State: {state}, Probability: {prob}")
    
    return measured_states

# Example usage: Try finding the marked state "11" in a database of size 4
N = 4  # Database size
marked_state = '11'  # Marked state to search for
grovers_algorithm(N, marked_state)


Grover's Algorithm Results:
State: |01>, Probability: 0.2724609375
State: |11>, Probability: 0.2646484375
State: |00>, Probability: 0.2197265625
State: |10>, Probability: 0.2431640625


[(|01>, 0.2724609375),
 (|11>, 0.2646484375),
 (|00>, 0.2197265625),
 (|10>, 0.2431640625)]

In [53]:
#Variational Quantum Eigensolvers (VQE) ALGORITHM

import numpy as np
from qat.lang.AQASM import Program, RX, RZ, CNOT
from qat.qpus import get_default_qpu
from scipy.optimize import minimize

# Define the number of qubits and parameters for VQE
num_qubits = 2
gamma = np.pi / 4  # Initial parameter value for rotations
beta = np.pi / 8   # Initial parameter value for rotations

# Create quantum program
prog = Program()
qbits = prog.qalloc(num_qubits)

# Ansatz: Apply rotations to qubits
def ansatz(prog, params):
    prog.apply(RX(params[0]), qbits[0])  # Apply RX to first qubit
    prog.apply(RZ(params[1]), qbits[1])  # Apply RZ to second qubit
    prog.apply(CNOT, qbits[0], qbits[1])  # Apply CNOT gate
    return prog

# Define Hamiltonian for the system (example Hamiltonian: X1 + X2)
def hamiltonian(prog):
    prog.apply(RX(np.pi), qbits[0])  # Example Hamiltonian component (Pauli X on qubit 0)
    prog.apply(RX(np.pi), qbits[1])  # Example Hamiltonian component (Pauli X on qubit 1)
    return prog

# Energy calculation (expectation value of the Hamiltonian)
def compute_energy(prog, qpu):
    # Convert the quantum program to a circuit and submit to QPU
    circuit = prog.to_circ()
    job = circuit.to_job(nbshots=1024)
    result = qpu.submit(job)
    
    # Calculate the expectation value of the Hamiltonian (energy)
    energy = 0
    for sample in result:
        # Sample contains measured states and their respective probabilities
        # Assuming each sample has a 'state' (binary string) and a 'probability'
        energy += sample.probability * compute_state_energy(sample.state)  # Placeholder for energy calculation
    return energy

# Function to compute energy based on the state (could be adjusted for your specific Hamiltonian)
def compute_state_energy(state):
    # Here you would define the energy contribution of each state based on your Hamiltonian
    # For simplicity, let's just return a simple energy based on state parity
    return 1 if state == '00' else -1  # Example: energy of 1 for |00> state, -1 for other states

# Energy function to minimize
def energy_function(params):
    prog = Program()
    ansatz(prog, params)  # Apply ansatz
    prog = hamiltonian(prog)  # Apply Hamiltonian
    qpu = get_default_qpu()  # Get default QPU
    energy = compute_energy(prog, qpu)
    return energy

# Use scipy's minimize function to optimize parameters
params = [gamma, beta]  # Initial guess for the parameters
optimized_params = minimize(energy_function, params, method='BFGS', options={'disp': True})

# Output optimized parameters and energy
print("Optimized Parameters:", optimized_params.x)
print("Final Energy:", energy_function(optimized_params.x))


Optimization terminated successfully.
         Current function value: -1.000000
         Iterations: 0
         Function evaluations: 3
         Gradient evaluations: 1
Optimized Parameters: [0.78539816 0.39269908]
Final Energy: -1.0


In [57]:
#Quantum Noise Simulation ALGORITHM

import numpy as np
from qat.lang.AQASM import Program, H, CNOT, X, Y, Z
from qat.qpus import get_default_qpu
import random

# Define the number of qubits
num_qubits = 2

# Create quantum program
prog = Program()
qbits = prog.qalloc(num_qubits)

# Apply Hadamard to the first qubit
prog.apply(H, qbits[0])

# Add depolarizing noise (example)
def add_noise(prog):
    # Randomly apply a Pauli noise operation (X, Y, Z) to qubit 0 with probability 0.1
    if random.random() < 0.1:
        # Apply depolarizing noise (X, Y, or Z gate)
        noise_gate = random.choice([X, Y, Z])
        prog.apply(noise_gate, qbits[0])
    return prog

# Define a simple circuit with noise
def noisy_circuit(prog):
    prog = add_noise(prog)  # Add noise to the program
    prog.apply(CNOT, qbits[0], qbits[1])  # Apply CNOT
    prog.measure(qbits[0])  # Measure qubit 0
    prog.measure(qbits[1])  # Measure qubit 1
    return prog

# Execute the noisy circuit
noisy_prog = noisy_circuit(prog)
circuit = noisy_prog.to_circ()
qpu = get_default_qpu()
job = circuit.to_job(nbshots=1024)
result = qpu.submit(job)

# Display results
for sample in result:
    print(f"State: {sample.state}, Probability: {sample.probability}")


State: |11>, Probability: 0.501953125
State: |00>, Probability: 0.498046875


In [59]:
#QCNN ALGORITHM
from qat.lang.AQASM import Program, H, CNOT, RX, RZ
from qat.qpus import get_default_qpu
import numpy as np

# Define a simple QCNN-like structure
def qc_nn_layer(prog, qbits, params):
    # Apply RX rotations and CNOT gates
    prog.apply(RX(params[0]), qbits[0])
    prog.apply(RZ(params[1]), qbits[1])
    prog.apply(CNOT, qbits[0], qbits[1])
    return prog

# Create a basic QCNN model
def qc_nn_model(prog, qbits, params):
    # Apply quantum convolutional layer
    prog = qc_nn_layer(prog, qbits, params)
    prog.measure(qbits[0])
    prog.measure(qbits[1])
    return prog

# Define the parameters
params = np.random.rand(2)  # Random initial parameters for QCNN layer
prog = Program()
qbits = prog.qalloc(2)

# Apply QCNN model
prog = qc_nn_model(prog, qbits, params)

# Convert to circuit and run
circuit = prog.to_circ()
qpu = get_default_qpu()
job = circuit.to_job(nbshots=1024)
result = qpu.submit(job)

# Display results
for sample in result:
    print(f"State: {sample.state}, Probability: {sample.probability}")


State: |00>, Probability: 0.9931640625
State: |11>, Probability: 0.0068359375


In [60]:
#HLL ALGORITHM
from qat.lang.AQASM import Program, RX, RZ, CNOT
from qat.qpus import get_default_qpu
import numpy as np

# Hybrid learning layer example
def hll_layer(prog, qbits, params):
    # Classical to Quantum data transfer
    prog.apply(RX(params[0]), qbits[0])  # Apply RX gate with parameter
    prog.apply(RZ(params[1]), qbits[1])  # Apply RZ gate with parameter
    prog.apply(CNOT, qbits[0], qbits[1])  # Apply CNOT gate
    return prog

# Define Hybrid Learning Layer for optimization
def hll_optimization(params):
    prog = Program()
    qbits = prog.qalloc(2)
    prog = hll_layer(prog, qbits, params)
    return prog

# Optimization using classical optimizer
params = np.random.rand(2)  # Random initial parameters for HLL layer
prog = hll_optimization(params)

# Convert to circuit and execute
circuit = prog.to_circ()
qpu = get_default_qpu()
job = circuit.to_job(nbshots=1024)
result = qpu.submit(job)

# Display results
for sample in result:
    print(f"State: {sample.state}, Probability: {sample.probability}")


State: |00>, Probability: 0.8369140625
State: |11>, Probability: 0.1630859375
